In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import re

def separate_integers_letters(string):
    integers = ''.join(re.findall(r'\d+', string))
    letters = ''.join(re.findall(r'[a-zA-Z]+', string))
    return integers, letters

In [3]:
dico_encodage = {'E_Bleu': 1, 'E_Rouge': 2, 'L_Bleu': 3, 'L_Rouge': 4}
print(dico_encodage)

{'E_Bleu': 1, 'E_Rouge': 2, 'L_Bleu': 3, 'L_Rouge': 4}


In [4]:
import pandas as pd

def add_labels(df):

  for index, row in df.iterrows():
    station_name_parts = row['station_name'].split('/')
    position_dynamique = station_name_parts[2]
    position_dynamique_parts = position_dynamique.split('_')
    _, position = separate_integers_letters(position_dynamique_parts[1])
    dynamique = position_dynamique_parts[2]

    position_dynamique = position + '_' + dynamique

    df.loc[index, 'label'] = dico_encodage[position_dynamique]

  return df

In [5]:
from sklearn.metrics import accuracy_score

def custom_cross_validation(model, X_train, y_train, X_test, y_test):
    # Entraîner le modèle sur l'ensemble de données d'entraînement
    model.fit(X_train, y_train)

    # Prédire les étiquettes sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Calculer la précision du modèle
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [6]:
import pandas as pd
import os
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Chemin vers le répertoire contenant les dossiers de données sur Google Drive
data_directory = "/content/gdrive/MyDrive/ListenToTheWild/day_temporal"

# Liste des dossiers dans le répertoire
directories = sorted(os.listdir(data_directory))

rf_mean_acc = 0
mlp_mean_acc = 0
gb_mean_acc = 0
# Boucle sur chaque dossier
for directory in directories:
  # Chemin vers le dossier contenant les fichiers de train/test
  directory_path = os.path.join(data_directory, directory)

  # Charger les données d'entraînement
  train_data = pd.read_csv(os.path.join(directory_path, "day_temporal_train.csv"))
  nb_rows_train = train_data.shape[0]
  # Ajouter les labels
  train_data_with_labels = add_labels(train_data)
  # Supprimer les colonnes "file_name" et "station_name"
  train_data_with_labels = train_data_with_labels.drop(['file_name', 'station_name'], axis=1)
  # Pour supprimer les NaN
  train_data_with_labels = train_data_with_labels.drop(['Ht', 'ACTtMean', 'TFSD'], axis=1)
  # Créer X et y
  X_train = train_data_with_labels.drop(columns=['label'])  # attributs
  y_train = train_data_with_labels['label']  # étiquettes

  # Charger les données de test
  test_data = pd.read_csv(os.path.join(directory_path, "day_temporal_test.csv"))
  nb_rows_test = test_data.shape[0]
  # Ajouter les labels
  test_data_with_labels = add_labels(test_data)
  # Supprimer les colonnes "file_name" et "station_name"
  test_data_with_labels = test_data_with_labels.drop(['file_name', 'station_name'], axis=1)
  # Pour supprimer les NaN
  test_data_with_labels = test_data_with_labels.drop(['Ht', 'ACTtMean', 'TFSD'], axis=1)
  # Créer X et y
  X_test = test_data_with_labels.drop(columns=['label'])  # attributs
  y_test = test_data_with_labels['label']  # étiquettes

  # Créer les modèles
  rf_model = RandomForestClassifier()
  mlp_model = MLPClassifier()
  gb_model = GradientBoostingClassifier()

  # Effectuer les validations croisées personnalisées
  rf_accuracy = custom_cross_validation(rf_model, X_train, y_train, X_test, y_test)
  mlp_accuracy = custom_cross_validation(mlp_model, X_train, y_train, X_test, y_test)
  gb_accuracy = custom_cross_validation(gb_model, X_train, y_train, X_test, y_test)

  # Calculer la moyenne des précisions
  mean_accuracy = (rf_accuracy + mlp_accuracy + gb_accuracy)/3

  # Ajouter les valeurs pour le calcul des moyennes
  rf_mean_acc += rf_accuracy
  mlp_mean_acc += mlp_accuracy
  gb_mean_acc += gb_accuracy

  # Calculer les pourcentages des ensembles
  per_train_set = nb_rows_train / (nb_rows_train + nb_rows_test)
  per_test_set = nb_rows_test / (nb_rows_train + nb_rows_test)

  # Afficher les précisions sur l'ensemble de test
  print(f"Dossier {directory} (train : {round(per_train_set*100, 2)}% / test : {round(per_test_set*100, 2)}%) :")
  print(f"Précision sur l'ensemble de test avec RF : {round(rf_accuracy, 2)}")
  print(f"Précision sur l'ensemble de test avec MLP : {round(mlp_accuracy, 2)}")
  print(f"Précision sur l'ensemble de test avec GB : {round(gb_accuracy, 2)}")
  print(f"Moyenne des précisions : {round(mean_accuracy, 2)}")
  print()

print("=======================================================================")
print()

# Afficher les moyennes des précisions par modèle
nb_dir = len(directories)
print(f"Moyenne des précisions avec RF : {round(rf_mean_acc/nb_dir, 2)}")
print(f"Moyenne des précisions avec MLP : {round(mlp_mean_acc/nb_dir, 2)}")
print(f"Moyenne des précisions avec GB : {round(gb_mean_acc/nb_dir, 2)}")

Dossier dossier_1 (train : 80.63% / test : 19.37%) :
Précision sur l'ensemble de test avec RF : 0.32
Précision sur l'ensemble de test avec MLP : 0.31
Précision sur l'ensemble de test avec GB : 0.34
Moyenne des précisions : 0.32

Dossier dossier_2 (train : 77.71% / test : 22.29%) :
Précision sur l'ensemble de test avec RF : 0.33
Précision sur l'ensemble de test avec MLP : 0.27
Précision sur l'ensemble de test avec GB : 0.35
Moyenne des précisions : 0.31

Dossier dossier_3 (train : 79.52% / test : 20.48%) :
Précision sur l'ensemble de test avec RF : 0.39
Précision sur l'ensemble de test avec MLP : 0.28
Précision sur l'ensemble de test avec GB : 0.34
Moyenne des précisions : 0.34

Dossier dossier_4 (train : 82.12% / test : 17.88%) :
Précision sur l'ensemble de test avec RF : 0.4
Précision sur l'ensemble de test avec MLP : 0.3
Précision sur l'ensemble de test avec GB : 0.39
Moyenne des précisions : 0.36

Dossier dossier_5 (train : 80.02% / test : 19.98%) :
Précision sur l'ensemble de test 